# Computing for Economists: Data Wrangling





In this module, we will have you work through the following datasets to analyze social connectedness in Europe. This module is inspired by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).

In [3]:
### Import Modules
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

## Tidying the Data

1. The `data` folder contains data from the following sources. Read each of the data files into Python as Pandas DataFrames.
  - https://ec.europa.eu/eurostat/en/web/products-datasets/-/EDAT_LFSE_04 
    - This data contains the percent of population by educational attainment level, sex, and NUTS 2 regions in the European Union.
  - https://ec.europa.eu/eurostat/web/products-datasets/-/nama_10r_2hhinc
    - The data contains the income of households by NUTS 2 region.  
  - https://ec.europa.eu/eurostat/web/products-datasets/product?code=demo_r_pjanaggr3
    - Population by age, group, sex, and NUTS 2 region.
  - https://ec.europa.eu/eurostat/web/gisco/geodata/reference-data/administrative-units-statistical-units/nuts
    - NUTS region shapefiles (use the geopandas library to read it as a GeoDataFrame).
  - https://media.githubusercontent.com/media/social-connectedness-index/euro_sci/master/_intermediate_data/geo_distance_dat.csv
    - Distance between pairs of NUTS 2 regions constructed by [Bailey et al. (2020)](https://pages.stern.nyu.edu/~jstroebe/PDF/BJRKSS_EuroSCI.pdf).
  - `gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv` from https://data.humdata.org/dataset/social-connectedness-index
    - This file contains the social connectedness index between NUTS 2 regions and all Global Administrative Areas (GADM). We will only consider NUTS 2 to NUTS 2 connections. Social connectedness is a snapshot of Facebook links as of July 2019.
    - Note: this file is very large so we recommend reading the file directly into the notebook at the following link: https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv


In [ ]:
eddf = pd.read_csv("data/estat_edat_lfse_04.tsv", sep="\t")
incomedf = pd.read_csv("data/estat_nama_10r_2hhinc.tsv", sep="\t")
popdf = pd.read_csv("data/estat_demo_r_pjanaggr3.tsv", sep="\t")
nuts_shapefiles = gpd.read_file("data/NUTS_RG_20M_2021_3035.shp")
dist_df = pd.read_csv(
    "https://media.githubusercontent.com/media/social-connectedness-index/euro_sci/master/_intermediate_data/geo_distance_dat.csv"
)
soc_con_url = "https://data.humdata.org/dataset/e9988552-74e4-4ff4-943f-c782ac8bca87/resource/5de92e01-606e-4e4d-ad7c-4a3d493a0cc3/download/gadm1_nuts2-gadm1_nuts2-fb-social-connectedness-index-october-2021.tsv"
sci_df = pd.read_csv(soc_con_url, sep="\t")


2. Clean the education data (`estat_edat_lfse_04.tsv`) and put it into "tidy" format. Check that there is a unique non-missing key for each row. The resulting data should have five columns and 299,088 rows.

In [50]:
# making copies so I don't have to redownload

eddf_clean = eddf.copy()
eddf_clean[["freq", "sex", "isced11", "age_group", "unit", "region"]] = eddf[
    "freq,sex,isced11,age,unit,geo\TIME_PERIOD"
].str.split(",", expand=True)

eddf_clean = (
    eddf_clean.query("sex=='T'")
    .drop(["freq", "sex", "unit", "freq,sex,isced11,age,unit,geo\TIME_PERIOD"], axis=1)
    .melt(
        id_vars=["region", "isced11", "age_group"], var_name="year", value_name="value"
    )
    .assign(
        value=lambda df: pd.to_numeric(df.value, errors="coerce"),
        year=lambda df: df["year"].astype(int),
    )
)
eddf_clean

,region,isced11,age_group,year,value
0,AT,ED0-2,Y20-24,2000,14.9
1,AT1,ED0-2,Y20-24,2000,15.3
2,AT11,ED0-2,Y20-24,2000,NaN
3,AT12,ED0-2,Y20-24,2000,14.8
4,AT13,ED0-2,Y20-24,2000,15.8
...,...,...,...,...,...
299083,UKM7,ED5-8,Y30-34,2023,NaN
299084,UKM8,ED5-8,Y30-34,2023,NaN
299085,UKM9,ED5-8,Y30-34,2023,NaN
299086,UKN,ED5-8,Y30-34,2023,NaN


3. Do the same for the income data (`estat_nama_10r_2hhinc.tsv`) and the population data (`estat_demo_r_pjanaggr3.tsv`).

In [51]:
income_df = incomedf.copy()
income_df[["frequency", "unit_type", "direction", "item", "region"]] = income_df[
    "freq,unit,direct,na_item,geo\TIME_PERIOD"
].str.split(",", expand=True)

income_df = (
    income_df.drop(columns=["frequency", "freq,unit,direct,na_item,geo\TIME_PERIOD"])
    .melt(
        id_vars=["unit_type", "direction", "item", "region"],
        value_name="amount",
        var_name="year",
    )
    .assign(
        amount=lambda df: pd.to_numeric(df["amount"], errors="coerce"),
        year=lambda df: df["year"].astype(int),
    )
)
income_df

,unit_type,direction,item,region,year,amount
0,EUR_HAB,BAL,B5N,AT,1995,NaN
1,EUR_HAB,BAL,B5N,AT1,1995,NaN
2,EUR_HAB,BAL,B5N,AT11,1995,NaN
3,EUR_HAB,BAL,B5N,AT12,1995,NaN
4,EUR_HAB,BAL,B5N,AT13,1995,NaN
...,...,...,...,...,...,...
326979,PPS_EU27_2020_HAB,BAL,B7N,LU0,2022,39100.0
326980,PPS_EU27_2020_HAB,BAL,B7N,LU00,2022,39100.0
326981,PPS_EU27_2020_HAB,BAL,B7N,LV,2022,NaN
326982,PPS_EU27_2020_HAB,BAL,B7N,LV0,2022,NaN


In [52]:
population_df = popdf.copy()
population_df[["frequency", "unit_type", "sex", "age_group", "region"]] = population_df[
    "freq,unit,sex,age,geo\TIME_PERIOD"
].str.split(",", expand=True)

population_df = (
    population_df.drop(columns=["frequency", "freq,unit,sex,age,geo\TIME_PERIOD"])
    .melt(
        id_vars=["unit_type", "sex", "age_group", "region"],
        value_name="population",
        var_name="year",
    )
    .assign(
        population=lambda df: pd.to_numeric(df["population"], errors="coerce"),
        year=lambda df: df["year"].astype(int),
    )
)

population_df.shape

(1061820, 6)

4. Outer join the percent of the population age 25 to 64 in a NUTS 2 region receiving a tertiary education with the income per capita in purchasing power standard (PPS). Then outer join the education and income data with population data for the total population (for sex and age). Only include data at the NUTS 2 level.

  To obtain the income per capita in PPS, the national account should be the "balance of primary incomes/national income, net" and the direction of flow should be the "balance".

In [63]:
merged_data = pd.merge(
    pd.merge(
        (
            eddf_clean.query("age_group == 'Y25-64' and isced11 == 'ED5-8'")
            .drop(columns=["age_group", "isced11"])
            .rename(columns={"value": "pct_ed_tertiary"})
        ),
        (
            income_df.query(
                "unit_type == 'PPS_EU27_2020_HAB' and item == 'B5N' and direction == 'BAL'"
            )
            .drop(columns=["unit_type", "item", "direction"])
            .rename(columns={"amount": "pps"})
        ),
        how="outer",
        on=["region", "year"],
    ),
    (
        population_df.query("age_group == 'TOTAL' and sex == 'T'").drop(
            columns=["unit_type", "sex", "age_group"]
        )
    ),
    how="outer",
    on=["region", "year"],
)

merged_data = merged_data.query("region.str.len() == 4").dropna().reset_index(drop=True)
merged_data

,region,year,pct_ed_tertiary,pps,population
0,AT11,2000,11.0,14800.0,276226.0
1,AT11,2001,11.2,14900.0,275956.0
2,AT11,2002,11.6,15400.0,276673.0
3,AT11,2003,12.6,16300.0,276542.0
4,AT11,2005,12.7,18600.0,278032.0
...,...,...,...,...,...
3486,SK04,2016,19.3,8500.0,1617347.0
3487,SK04,2017,20.1,8300.0,1620413.0
3488,SK04,2018,23.6,8800.0,1623043.0
3489,SK04,2019,23.8,9200.0,1625436.0


5. For each pair of NUTS 2 regions, obtain the absolute difference in the two variables constructed in problem 4 in 2019. Drop the French islands (starts with NUTS 2 region code "FRY").


In [66]:
from itertools import combinations

df = merged_data.query("year==2019 & ~region.str.startswith('FRY')")
region_pairs = list(combinations(df["region"].unique(), 2))
# Initialize a list to store the results
diff_list = []

# Calculate the absolute differences for each pair
for reg1, reg2 in region_pairs:
    data_reg1 = df[df["region"] == reg1]
    data_reg2 = df[df["region"] == reg2]

    diff_list.append(
        {
            "region_1": reg1,
            "region_2": reg2,
            "pct_ed_tertiary": abs(
                data_reg1["pct_ed_tertiary"].values[0]
                - data_reg2["pct_ed_tertiary"].values[0]
            ),
            "pps": abs(data_reg1["pps"].values[0] - data_reg2["pps"].values[0]),
            "population": abs(
                data_reg1["population"].values[0] - data_reg2["population"].values[0]
            ),
        }
    )

# Create a DataFrame to store the results
diff_df = pd.DataFrame(diff_list)

diff_df.head()

,region_1,region_2,pct_ed_tertiary,pps,population
0,AT11,AT12,1.7,1600.0,1384109.0
1,AT11,AT13,12.2,700.0,1604058.0
2,AT11,AT21,0.1,1100.0,267506.0
3,AT11,AT22,0.6,100.0,949619.0
4,AT11,AT31,1.1,1400.0,1188662.0


6. Now we will merge the the 2019 data with the SCI data and the pairwise distance data. Create a dataframe with one row for each pair of NUTS 2 regions (as two separate columns) and the following variables:

*   Log scaled SCI
*   Log distance (add 1 to avoid zeros)
*   Percent tertiary education for User and Friend NUTS 2
*   Income per capita for User and Friend NUTS 2
*   Population for User and Friend NUTS 2
*   Absolute difference in tertiary education between User and Friend NUTS 2
*   Absolute difference in income per capita between User and Friend NUTS 2

  Note that the SCI data contains GADM areas, please only keep the NUTS 2 to NUTS 2 rows. Keeps all rows in the SCI data.


In [82]:
# Filter SCI data to only include NUTS 2 regions
sci_df = sci_df[(sci_df["user_loc"].str.len() == 4) & (sci_df["fr_loc"].str.len() == 4)]

# Merge dataframes
merged_df = (
    sci_df.merge(
        df[["region", "pct_ed_tertiary", "pps", "population"]],
        how="inner",
        left_on="user_loc",
        right_on="region",
    )
    .rename(
        columns={
            "pct_ed_tertiary": "pct_ed_user",
            "pps": "pps_user",
            "population": "pop_user",
        }
    )
    .merge(
        df[["region", "pct_ed_tertiary", "pps", "population"]],
        how="inner",
        left_on="fr_loc",
        right_on="region",
    )
    .rename(
        columns={
            "pct_ed_tertiary": "pct_ed_friend",
            "pps": "pps_friend",
            "population": "pop_friend",
        }
    )
    .merge(dist_df, how="inner", on=["user_loc", "fr_loc"])
    .assign(
        log_scaled_sci=lambda x: np.log1p(x["scaled_sci"]),
        log_distance=lambda x: np.log1p(x["distance"]),
        abs_diff_tertiary_edu=lambda x: abs(x["pct_ed_user"] - x["pct_ed_friend"]),
        abs_diff_income_per_capita=lambda x: abs(x["pps_user"] - x["pps_friend"]),
    )
)

result = merged_df[
    [
        "user_loc",
        "fr_loc",
        "log_scaled_sci",
        "log_distance",
        "pct_ed_user",
        "pct_ed_friend",
        "pps_user",
        "pps_friend",
        "pop_user",
        "pop_friend",
        "abs_diff_tertiary_edu",
        "abs_diff_income_per_capita",
    ]
]

result.head()

,user_loc,fr_loc,log_scaled_sci,log_distance,pct_ed_user,pct_ed_friend,pps_user,pps_friend,pop_user,pop_friend,abs_diff_tertiary_edu,abs_diff_income_per_capita
0,AT11,AT11,16.548203,0.000000,30.7,30.7,24300.0,24300.0,293433.0,293433.0,0.0,0.0
1,AT11,AT12,13.256098,4.588326,30.7,32.4,24300.0,25900.0,293433.0,1677542.0,1.7,1600.0
2,AT11,AT13,12.886826,4.331969,30.7,42.9,24300.0,23600.0,293433.0,1897491.0,12.2,700.0
3,AT11,AT21,11.830280,5.393574,30.7,30.8,24300.0,23200.0,293433.0,560939.0,0.1,1100.0
4,AT11,AT22,13.332713,4.733167,30.7,30.1,24300.0,24200.0,293433.0,1243052.0,0.6,100.0


## Descriptive Statistics


1.   Each country is represented by the first two letters of the NUTS 2 index. Present a country-level population-weighted summary table for the percent people with tertiary education and income per capita in 2019. Include the mean, standard deviation, min, and max. Round all numeric outputs to two decimal places.

In [68]:
(
    df.assign(country_code=lambda df: df["region"].str[:2])
    .groupby("country_code")
    .agg(
        {
            "pct_ed_tertiary": ["mean", "std", "min", "max", "count"],
            "pps": ["mean", "std", "min", "max", "count"],
        }
    )
)

pct_ed_tertiary                                        pps  \
                        mean        std   min   max count          mean   
country_code                                                              
AT                 32.200000   4.233793  28.4  42.9     9  25222.222222   
BE                 41.400000   6.756034  32.0  55.7    11  23354.545455   
BG                 25.616667   7.715806  19.2  40.5     6   8400.000000   
CY                 44.700000        NaN  44.7  44.7     1  17900.000000   
CZ                 23.825000   9.278816  15.2  45.5     8  16600.000000   
DK                 37.420000   9.228055  30.3  52.7     5  21700.000000   
EE                 39.500000        NaN  39.5  39.5     1  14100.000000   
EL                 27.207692   5.356532  20.6  40.2    13  12138.461538   
ES                 36.968421   7.695601  22.1  50.8    19  17068.421053   
FI                 43.040000   6.888614  34.8  53.7     5  20220.000000   
FR                 34.319048   6.001301  27.1  51.5    21  19052.380952   
HR                 25.825000  12.467391  14.9  42.8     4  11750.000000   
HU                 24.325000   9.905085  17.0  47.5     8  12950.000000   
LT                 46.750000  12.940054  37.6  55.9     2  17600.000000   
LU                 47.000000        NaN  47.0  47.0     1  30900.000000   
LV                 35.700000        NaN  35.7  35.7     1  12500.000000   
NO                 39.133333   6.066575  32.5  44.4     3  21633.333333   
PL                 31.100000   7.258616  24.2  56.1    17  13870.588235   
PT                 23.028571   6.432655  13.8  35.2     7  14371.428571   
RO                 18.450000   9.177301  11.7  40.3     8  12225.000000   
SE                 41.387500   5.967157  33.6  53.0     8  20475.000000   
SI                 33.600000   7.212489  28.5  38.7     2  16600.000000   
SK                 28.675000  11.236065  22.2  45.5     4  13550.000000   

                                                   
                      std      min      max count  
country_code                                       
AT            1555.456346  23200.0  28200.0     9  
BE            3355.402699  18200.0  29400.0    11  
BG            3557.527231   6100.0  15600.0     6  
CY                    NaN  17900.0  17900.0     1  
CZ            2678.485713  14500.0  22500.0     8  
DK            2366.431913  20000.0  25800.0     5  
EE                    NaN  14100.0  14100.0     1  
EL            2245.936502  10000.0  16500.0    13  
ES            3367.005368  12600.0  24700.0    19  
FI            3166.543857  17300.0  24400.0     5  
FR            2583.721937  16500.0  29200.0    21  
HR            3246.023208   8600.0  16300.0     4  
HU            3213.809489  10100.0  20100.0     8  
LT            4384.062043  14500.0  20700.0     2  
LU                    NaN  30900.0  30900.0     1  
LV                    NaN  12500.0  12500.0     1  
NO            1285.820101  20700.0  23100.0     3  
PL            2990.770115  11100.0  24100.0    17  
PT            2149.196971  12600.0  18100.0     7  
RO            5835.543799   7800.0  26000.0     8  
SE            2693.775895  18400.0  26700.0     8  
SI            1131.370850  15800.0  17400.0     2  
SK            5615.158057   9200.0  21800.0     4

2. The SCI is scaled from 1 to 1,000,000 and measures the relative probability of a Facebook friendship link ([source](https://data.humdata.org/dataset/social-connectedness-index?)). If a number is two times higher, then a friendship link between two users from those NUTS 2 regions is twice as likely. Calculate  the mean, std, min, and max of log(scaled SCI) by country and merge it with the summary table from the previous part.

3. Compute the correlation between log(scaled SCI), log(distance), user region education and income, as well as differences in education and income between NUTS 2 regions. Is it higher or lower than what you expected?

## Data Visualization

1. We will ask you to plot two maps of social connections of (1) a country and (2) a NUTS 2 region to all other NUTS 2 regions. Remember the French islands should be dropped (these islands are far away from continental Europe and will therefore make the maps harder to visualize).

  * For the chosen country, aggregate the SCI to produce country to NUTS 2 region measures. Scaled SCI can be aggregated correctly using population shares following [SCI documentation](https://data.humdata.org/dataset/social-connectedness-index). For countries $i, j$ with regions $\{r_i\}, \{r_j\}$, aggregate SCI is given by:

  $$SCI_{i,j} = \sum_{\{r_i\}}\sum_{\{r_j\}} PopShare_{r_i} \times PopShare_{r_j} \times SCI_{r_i, r_j}$$

  where $PopShare_{r_i}$ is the share of country $i$'s population in region $r_i$.

  * Merge the aggregated SCI data with the GeoDataFrame (keep only NUTS 2 regions) for the applicable geographies.
  * Construct the bins for the map legend by first selecting a baseline SCI cutoff (e.g. 20th percentile of the **total data** that you are working with). Then calculate the subsequent cutoffs as $2\times$, $3\times$, $5\times$, $10\times$, $25\times$, $100\times, 1000\times$ that of the baseline cutoff. Feel free to use another legend system as long as there is justification.
  * Map directly using the GeoPandas library, or feel free to explore interactive maps in Plotly.

  Recall the principles of software engineering (consider writing a function for code that may be reused).

2. Show the relationship between log SCI and log distance in a figure. Highlight the points of at least one user geography (country or NUTS 2). Fit a curve to the data and plot it along with the scatterplot.

3. Visualize the distributions of log SCI by income bin (e.g. low, medium, high) of the user region. Comment.

Bonus: Explore the data and feel free to visualize whatever you would like. Make sure you understand the variables and compute summary statistics.

# Submission ⭐

Congratulations you finished Module 2! We will handle all exercise submissions on GitHub Classroom. Please push your changes to your team repository folder on GitHub Classroom.